In [ ]:
%gui qt

Same exercise, (data, normalization function) as io but with interaction

# Create two functions

![ImageDialog](img/ImageDialog.png)

- `getFlatfield`: to select the background
- `getBackground`: to select the flatfield

To do this use the ImageFileDialog class

- see doc: http://www.silx.org/doc/silx/latest/modules/gui/dialog/imagefiledialog.html
- related example: examples/fileDialog.py

In [ ]:
from silx.gui.dialog.ImageFileDialog import ImageFileDialog

In [ ]:
import os
def getImage(title=None):
    dialog = ImageFileDialog()
    dialog.setWindowTitle(title or 'Get image')
    dir_ = 'data'
    assert os.path.isdir(dir_)
    dialog.setDirectory(dir_)
    if not dialog.exec_():
        return None
    else:
        return dialog.selectedImage()

In [ ]:
def getFlatfield():
    return getImage('Select flat field')

In [ ]:
def getBackground():
    return getImage('Select the background image')

# Create a function to correct an image from flatfield and background

take as input an image, background and flatfield. Return the normalized image

In [ ]:
def normalized(image, flatfield, background):
    return (image - background) / (flatfield - background)

# Select an image and display it raw and normalized

In [ ]:
from silx.gui.plot import Plot2D
def showRawAndCorrected(raw, corrected):
    plot = Plot2D()
    plot.addImage(data=raw, replace=False, legend='raw')
    plot.addImage(data=corrected, replace=False,
                  legend='corrected', origin=(raw.shape[0], 0))
    plot.show()

In [ ]:
image = getImage()
background = getBackground()
flatfield = getFlatfield()
if image is not None and background is not None and flatfield is not None:
    corrected = normalized(image=image,
                           flatfield=flatfield,
                           background=background)
    showRawAndCorrected(raw=image, corrected=corrected)

# Add an action to apply the correction on a stack of image

![ImageDialog](img/svNormalization.png)

Here is the sample code to plot the stack of image

In [ ]:
from silx.gui.plot.StackView import StackViewMainWindow
import h5py
import numpy

dataFile = h5py.File('data/ID16B_diatomee.h5')

mystack = numpy.array([dataFile['data']['0000'][...],
                       dataFile['data']['0100'][...],
                       dataFile['data']['0200'][...],
                       dataFile['data']['0299'][...],
                       dataFile['data']['0400'][...],
                       dataFile['data']['0500'][...]])

sv = StackViewMainWindow()
sv.setStack(mystack)
sv.show()

Here is a function to apply the corection on the stack

In [ ]:
def applyCorrection(images, flatfield, background):
    correctedImgs = []
    for image in images:
        correctedImgs.append(normalized(image,
                                        flatfield=flatfield,
                                        background=background))
    return correctedImgs

To define an action:
    - heritate from PlotAction
    - redefine the triggered function
    - See the tutorial on how to add an action see: http://www.silx.org/doc/silx/dev/modules/gui/plot/actions/examples.html
    - you can also use the PlotAction tutorial.ipynb

In [ ]:
from silx.gui.plot.actions import PlotAction 
class CorrectImageAction(PlotAction):
    """QAction applying the correction algorithm

    :param plot: :class:`.PlotWidget` instance on which to operate
    :param parent: See :class:`QAction`
    """
    def __init__(self, plot, sv, parent=None):
        PlotAction.__init__(self,
                            plot,
                            icon='shape-circle',
                            text='process correction',
                            tooltip='apply the correction algorithm',
                            triggered=self._actionTriggered,
                            parent=parent,
                           checkable=False)
        self.stackViewer = sv

    def _actionTriggered(self):
        flatfield = getFlatfield()
        background = getBackground()
        if flatfield is None or background is None:
            return
        newStack = applyCorrection(images=self.stackViewer.getStack()[0],
                                   flatfield=flatfield,
                                   background=background)
        self.stackViewer.setStack(newStack)

In [ ]:
sv = StackViewMainWindow()
sv.setStack(mystack)

toolBar = sv.getPlot().toolBar()
myaction = CorrectImageAction(sv.getPlot(), sv)
toolBar.addAction(myaction)
sv.show()